In [1]:
import pandas as pd
import re
import numpy as np
df = pd.read_csv('LikelihoodRatiosDataFrame_GLR/DxMxLR.csv', index_col=0)

In [2]:
df.head()

,Association,Dx Name,Mx Name,Prevalence,Import,LR (+),LR (-)
0,"362, 59",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,0.5,2.0,1.500000,0.5000
1,"362, 62",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,0.5,2.0,0.666667,1.2000
2,"362, 64",PEPTIC ULCER,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,0.5,2.0,0.666667,2.0000
3,"362, 198",PEPTIC ULCER,AGE 16 TO 25,0.5,3.0,0.000100,-0.0003
4,"362, 199",PEPTIC ULCER,AGE 26 TO 55,0.5,3.0,0.000100,-0.0001


In [37]:
Mx_list = ['+AGE 16 TO 25', '+ABDOMEN PAIN COLICKY', '+JAUNDICE']
Dx_list = pd.DataFrame(columns=['Rank', 'Dx', 'Score', 'Importance_List'])
Dx_list['Dx']=pd.Series((list(set(df['Dx Name']))))
Dx_list['Importance_List']=pd.Series( [ [] for i in range(10) ])

In [38]:
for i,r in Dx_list.iterrows():
    #r['Dx'] = disease
    print(r['Dx'])
    score = 1
    for mx in Mx_list: 
        if re.match('[+]', mx): 
            mx = mx.strip('+')
            q = df.loc[df['Dx Name'] == r['Dx']]
            q = q.loc[df['Mx Name'] == mx]
            print(q)
            im = q['Import'].values
            if im > 3:
                Dx_list.loc[i, 'Dx'] = str(Dx_list.loc[i, 'Dx'] + '*')
            p = q['LR (+)'].values
            if p.size == 0: continue #don't count NAs
            else:
                score = score * p
                Dx_list.loc[i,'Importance_List'].append(float(im))
        elif re.match('[-]', mx): 
            mx = mx.strip('-')
            s = df.loc[df['Dx Name'] == r['Dx']]
            s = s.loc[df['Mx Name'] == mx]
            im = s['Import'].values
            m = s['LR (-)'].values
            if m.size == 0: continue #don't count NAs
            else: 
                score = score * m
                Dx_list.loc[i,'Importance_List'].append(float(im))
        else: print("Mx ", mx, "is not positive or negative. Please indicate presence or absence of mx.")
    if not Dx_list.loc[i,'Importance_List']: 
        Dx_list.loc[i, 'Score'] = 0
    else:
        Dx_list.loc[i, 'Score'] = float(score)

PANCREATITIS ACUTE
   Association             Dx Name       Mx Name  Prevalence  Import  LR (+)  \
51    351, 198  PANCREATITIS ACUTE  AGE 16 TO 25         0.1     3.0  0.0009   

      LR (-)  
51  1.124873  
Empty DataFrame
Columns: [Association, Dx Name, Mx Name, Prevalence, Import, LR (+), LR (-)]
Index: []
    Association             Dx Name   Mx Name  Prevalence  Import  LR (+)  \
117   351, 2026  PANCREATITIS ACUTE  JAUNDICE         0.1     5.0    2.25   

      LR (-)  
117  0.84375  
PEPTIC ULCER
  Association       Dx Name       Mx Name  Prevalence  Import  LR (+)  LR (-)
3    362, 198  PEPTIC ULCER  AGE 16 TO 25         0.5     3.0  0.0001 -0.0003
Empty DataFrame
Columns: [Association, Dx Name, Mx Name, Prevalence, Import, LR (+), LR (-)]
Index: []
Empty DataFrame
Columns: [Association, Dx Name, Mx Name, Prevalence, Import, LR (+), LR (-)]
Index: []
GASTROINTESTINAL SARCOIDOSIS
    Association                       Dx Name       Mx Name  Prevalence  \
585    192, 198  GASTRO

In [34]:
Dx_list = Dx_list.sort_values(by="Score", ascending= False)

In [35]:
rank = 1
for i,r in Dx_list.iterrows():
    Dx_list.loc[i, 'Rank'] = rank
    rank = rank + 1
Dx_list.set_index('Rank', inplace=True)
Dx_list

,Dx,Score,Importance_List
Rank,,,
1,CHOLEDOCHOLITHIASIS*,107811,"[2.0, 3.0, 5.0]"
2,ANISAKIASIS,24999.8,[3.0]
3,GASTROINTESTINAL SARCOIDOSIS,2499.75,[3.0]
4,PYLORIC OBSTRUCTION,2499.75,[3.0]
5,PANCREATITIS ACUTE*,2.25,[5.0]
6,PEPTIC ULCER,0,[]
7,GASTRITIS GIANT HYPERTROPHIC <MENETRIERS>,0,[]
8,PANCREATITIS CHRONIC,0,[]
9,GASTRIC LYMPHOMA,0,[]
